In [9]:
import psycopg2 as pg2
import pandas as pd
# pd.set_option('display.expand_frame_repr', False)
# pd.set_option('display.max_colwidth', -1)
import pprint 
import string
import unicodedata
import numpy as np
import pickle

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import decomposition

import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.util import ngrams
from nltk import pos_tag
from nltk import RegexpParser
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

## Import pickled cleaned data from previous project

In [10]:
with open('../data/boardgamedata.pickle', 'rb') as f:
    worddata, numdata = pickle.load(f)
    
worddata.reset_index(drop=True, inplace=True)


In [117]:
special = ['&quot;', '&#10;']
for chara in special:
    worddata['description']= worddata['description'].apply(lambda x: x.replace(chara,''))

In [130]:
with open('../data/boardgamedata.pickle', 'wb') as f:
    pickle.dump([worddata, numdata], f)
    

In [118]:
worddata[10:15]

,game.id,name,description,category,mechanic
10,10,Elfenland,"Elfenland is a redesign of the original White Wind game Elfenroads. The game is set in the mythical world of the elves. A group of fledgling elves (the players) are charged with visiting as many of the twenty Elfencities as they can over the course of 4 rounds. To accomplish the task they will use various forms of transportation such as Giant Pigs, Elfcarts, Unicorns, Rafts, Magic Clouds, Trollwagons, and Dragons.Gameplay: Players begin in the Elf capitol, draw one face down movement tile, and are dealt eight transport cards and a secret 'home' city card that they must reach at the end of the 4th round or lose points for each city space away from 'home' they are at the end of the game. Markers of each player's color are placed in each city on the board and are collected when the player visits that city (each counts as 1 point). The round proceeds in 2 stages. The first part of the round consists of the drawing of Tiles showing the differing types of transport (except rafts) from a combination of face up and face down tiles (if a player doesn't like the 5 tiles that are face up; they can always draw blind from the face down tiles and hope to get one they need). These transport tiles need to match the Transportation cards in your hand to use them most effectively. After each player has a total of 4 tiles they take turns placing a tile on any one of the roads that run between the elf cities. Only one transport tile may be placed on each road; so players may use other players tiles to travel if they have the matching cards in their hand. This frequently causes a readjustment of planned travel routes as other players tiles can allow you to move farther or shorter than you had first thought. Players can play their tiles to help themselves or hinder others by playing a slow mode of transport on another players (perceived) path.Each mode of transport has certain terrain it can travel through quickly or slowly, and those that it cannot. These are listed on the top of each transportation card by the number terrain symbols. The number of terrain symbols equals how many matching cards you must play to move across a given tile in a given terrain. For example, a Magic Cloud tile placed in a mountain would take one Magic cloud card to travel across (1 mountain symbol on card means Magic clouds are fast in mountains). If the same tile was placed on a road in forest terrain it would require 2 Magic Cloud cards to travel that route (2 Forest symbols on card means Magic Clouds are slow in Forest). Magic Clouds cannot travel in desert terrain at all (no desert symbols on card). All modes of transport are different and Rafts can be used on rivers or lakes without needing tiles. Rafts go slow upstream (2 raft cards needed) and fast downstream (1 card needed). The small lake requires 1 raft card to travel across and the larger lake requires 2 cards to travel across. Players may keep one unused transport counter and up to 4 Transportation cards from one round to the next.The second part of the round begins after all players have finished placing their transportation tiles for the round. Each player plays his cards and moves his elf-boot around the board collecting his tokens from the cities visited. If there is a Transport tile on a route and a player has no matching Transportation card he may 'Caravan' across it by playing any 3 Transportation cards from his hand.As a bit of 'take that' each player has a trouble tile which can be placed next to any transportation tile during the first part of the round. This counter means that in order to travel that path an additional card of the transport type must be played or 4 cards to 'Caravan'.Victory: if at the end of round 3 a player has visited all 20 cities he is the winner. If not the game ends after round 4 when 'Home' cities are revealed and each player subtracts points for each city he is away from his 'home' subtracting that from his collec

## Build LDA Model

In [119]:
#define common words in data to remove
#built from pervious project
sw = set(stopwords.words('english'))

COMMON_WORDS = ["game","point","board","player","rule","turn",
                "tile","card","deck","hand","points","victory",
                "win","lose","defeat","quot","players","play"
               ]
sw.update(COMMON_WORDS)

#### Remove numbers from vectorizer

Implement preprocess function

In [137]:
def preprocess_text(text):
    
#if text is empty, return empty bag of words
    if (text == None):
        return []

    if (len(text) < 1):
        return []
#remove numbers
    text = text.lower()
    text = re.sub(r'\d+', '', text)

#define common words in data to remove
    COMMON_WORDS = ["game","point","board","player","rule","turn",
                "tile","card","deck","hand","points","victory",
                "win","lose","defeat","players","play"]
    
# #removes common words
#     for word in text.split():
#         if word in COMMON_WORDS:
#             text = text.replace(word, "")

#tokenize sentences and add part of speech tag
    sent_tokens = sent_tokenize(text)

    tokens = list(map(word_tokenize, sent_tokens))

    sent_tags = list(map(pos_tag, tokens))

    grammar = r"""
        SENT: {<(J|N).*>}    """

    cp = RegexpParser(grammar)
    ret_tokens = list()
    stemmer_snowball = SnowballStemmer('english')
    lemmatizer = WordNetLemmatizer()

    for sent in sent_tags:
        tree = cp.parse(sent)
        for subtree in tree.subtrees():
            if subtree.label() == 'SENT':
                t_tokenlist = [tpos[0].lower() for tpos in subtree.leaves()]
                
                t_tokens_lemmatize = list(map(lemmatizer.lemmatize, t_tokenlist))
                t_tokens_stemsnowball = list(map(stemmer_snowball.stem, t_tokens_lemmatize))
                ret_tokens.extend(t_tokens_stemsnowball)
    ret = []

#runs through the ret_tokens and compare with common words again and removes common words
    for token in ret_tokens:
        if token in COMMON_WORDS:
            pass
        else:
            ret.append(token)
    return(' '.join(ret))

In [138]:
count_vectorizer = CountVectorizer(stop_words=sw, 
                                   preprocessor=preprocess_text, 
                                   max_df=0.85,
                                  min_df=2, 
                                   max_features=1000,
                                  strip_accents='unicode')

In [139]:
word_vec = count_vectorizer.fit_transform(worddata['description'])

/home/zy/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ourselv', 'sha', 'victori', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [140]:
lda = LatentDirichletAllocation(learning_method='online', n_jobs=-2, 
                                random_state=1659)
lda.fit(word_vec)

LatentDirichletAllocation(learning_method='online', n_jobs=-2,
                          random_state=1659)

In [21]:
def top_topic_features(model, feature_names, num_features=10):
    sorted_topics = feature_names[model.components_.argsort(axis=1)[:, ::-1][:, :num_features]]
    return sorted_topics

In [141]:
feature_names = np.array(count_vectorizer.get_feature_names())
top_topic_features(lda, feature_names=feature_names)


array([['number', 'round', 'first', 'th', 'score', 'color', 'token',
        'valu', 'end', 'anim'],
       ['dice', 'die', 'question', 'roll', 'child', 'parti', 'answer',
        'challeng', 'eacut', 'fun'],
       ['hero', 'ship', 'new', 'star', 'king', 'dragon', 'planet',
        'dungeon', 'pack', 'expans'],
       ['citi', 'race', 'track', 'money', 'phase', 'car', 'zombi',
        'monopoli', 'train', 'cube'],
       ['team', 'word', 'letter', 'british', 'aacut', 'block', 'sid',
        'la', 'que', 'six'],
       ['battl', 'war', 'unit', 'scenario', 'combat', 'armi', 'map',
        'forc', 'th', 'counter'],
       ['new', 'rsquo', 'expans', 'rdquo', 'ldquo', 'edit', 'bull',
        'action', 'set', 'compani'],
       ['charact', 'action', 'time', 'power', 'world', 'monster', 'way',
        'build', 'differ', 'good'],
       ['piec', 'space', 'oppon', 'move', 'squar', 'pawn', 'side',
        'number', 'th', 'first'],
       ['ndash', 'red', 'black', 'stone', 'treasur', 'color', 'b

### Create Probability Matrix

In [23]:
def predict_proba(model, vectorizer, text):
    if type(text) == str:
        text = [text]
    vec_text = vectorizer.transform(text)
    doc_probs = model.transform(vec_text)
    return doc_probs

In [142]:
game_probs = predict_proba(lda, count_vectorizer, worddata.description)

In [143]:
#pickle the fitted transformed model

with open('../models/lda.pickle', 'wb') as f:
    pickle.dump([lda,count_vectorizer,game_probs], f)

## Building the Recommender

In [22]:
# function to find index of FIRST  game given the game title

def find_game_idx(df, game_title):
    try:
        return df.name[df.name.str.contains(game_title)].index[0]
    except:
        return None

def find_game_id(df, idx):
    return df['game.id'].iloc[idx]


## TODO: if search returns multiple results, what then?? Need to solve this

In [23]:
search = worddata.name.str.contains('Betrayal')
search.sum()


11

## /TODO

In [25]:
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances

In [26]:
def sort_by_distance(idx, probs, distance_funct=cosine_distances, num_games=10):
    #returns array of indices of games closest to the provided game index
    return distance_funct(probs[idx].reshape(1,-1), probs).ravel().argsort()[:10]

* Debug sort_by_distance function
* check distances inside the function


In [27]:
def find_closest_games_names(sorted_distances, names):
    #finds names of an array of game idx's
    name_array = names.iloc[sorted_distances]
    return name_array

In [132]:
def recommend(df, game_title, prob_matrix, 
             distance_funct=cosine_distances, num_games=10):
    game_index = find_game_idx(df, game_title)
    game_sim = sort_by_distance(game_index, prob_matrix, distance_funct, num_games)
    game_recs = find_closest_games_names(game_sim, df.name)
    game_ids = find_game_id(df, game_rec.index)
    
#     rec_df = pd.DataFrame({'id':game_ids,'name':game_recs})
    rec_df = find_closest_games(game_sim,df)
    
    return rec_df

In [29]:
game_rec = recommend(worddata, 'Catan', game_probs)
game_rec

NameError: name 'game_probs' is not defined

In [126]:
def find_closest_games(sorted_distances, gamedata_df):
    #finds names of an array of game idx's
    name_array = gamedata_df.iloc[sorted_distances]
    return name_array

In [151]:
#save prob matrix for use later
with open('../models/prob_matrix.pickle', 'wb') as f:
    pickle.dump(game_probs,f)

## TODO: use suprise Library to test

In [165]:
worddata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74264 entries, 0 to 74263
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   game.id      74264 non-null  int64 
 1   name         74264 non-null  object
 2   description  74264 non-null  object
 3   category     74264 non-null  object
 4   mechanic     74264 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.8+ MB


In [146]:
def load_data():
    with open('../data/boardgamedata.pickle', 'rb') as f:
        worddata, numdata = pickle.load(f)
    worddata.reset_index(drop=True, inplace=True)

    return worddata, numdata

def load_model():
    with open('../models/prob_matrix.pickle', 'rb') as f:
        game_probs = pickle.load(f)
    return game_probs

def find_game_idx(df, game_title):
    #returns the first found index of a game given a name search string query
    return df.name[df.name.str.contains(game_title)].index[0]


def find_game_id(df, idx):
    #returns game id of an array of indices of games
    return df['game.id'].iloc[idx]


def sort_by_distance(idx, probs, num_games):
    #returns array of indices of games most similiar to the provided game index
    return cosine_distances(probs[idx].reshape(1,-1), probs).ravel().argsort()[:num_games]

def find_closest_games_names(sorted_distances, names):
    #finds names of an array of game idx's
    name_array = names.iloc[sorted_distances]
    return name_array

def find_closest_games(sorted_distances, gamedata_df):
    #finds names of an array of game idx's
    name_array = gamedata_df.iloc[sorted_distances]
    return name_array

def recommend_games(game_title, num_games):
    #returns recommendations for games in a list format
    df, numdata = load_data()
    prob_matrix = load_model()
    
    game_index = find_game_idx(df, game_title)
    game_sim = sort_by_distance(game_index, prob_matrix, num_games)
    game_recs = find_closest_games_names(game_sim, df.name)
    game_ids = find_game_id(df, game_recs.index)
    
    rec_df = find_closest_games(game_sim,df)
    to_html = rec_df.drop(axis=1,columns='game.id')
    to_html['category'] = to_html.category.apply(lambda x: ",".join(x))
    to_html['mechanic'] = to_html.mechanic.apply(lambda x: ",".join(x))
    to_html['short_desc'] = to_html.description.apply(lambda x: ' '.join(re.split(r'(?<=[.:;])\s', x)[:4]))
    to_html['short_desc'] = to_html['short_desc'].apply(lambda x: lxml.html.fromstring(x).text_content())
    to_html.drop(axis=1,columns='description',inplace=True)

    return to_html

In [154]:
data = recommend_games("Catan", 15)

In [155]:
data.head()


,name,category,mechanic,short_desc
12,Catan,Negotiation,"Dice Rolling,Hand Management,Modular Board,Route/Network Building,Trading","In Catan (formerly The Settlers of Catan), players try to be the dominant force on the island of Catan by building settlements, cities, and roads. On each turn dice are rolled to determine what resources the island produces. Players collect these resources (cards)&mdash;wood, grain, brick, sheep, or stone&mdash;to build up their civilizations to get to 10 victory points and win the game.Setup includes randomly placing large hexagonal tiles (each showing a resource or the desert) in a honeycomb shape and surrounding them with water tiles, some of which contain ports of exchange. Number disks, which will correspond to die rolls (two 6-sided dice are used), are placed on each resource tile."
36033,Humans!!! 2: Sea Food,"Expansion for Base-game,Fighting,Horror,Humor","Hand Management,Roll,Spin and Move,Variable Player Powers","Twilight Creations is pleased to announce our first expansion for our popular Humans!!! line. Humans!!! 2: Sea Food is set on a naval base and features 16 new event cards, 16 new tiles, rules and 10 new zombie navy figure pawns with player card. Map Tile art is by Kurt Miller, and Event card art is by fan favorite David Aikins.Part of the Zombies!!! series."
37737,Geckopoly,"Economic,Negotiation","Auction/Bidding,Roll,Spin and Move,Set Collection,Trading","Geico Insurance, the company known for its great rates, superior service, and, of course, its talkative gecko mascot, created this custom game to reward its agents for meeting and exceeding sales goals. A promotional game, yes, but the omnipresent gecko made it a popular purchase at Geico company store sites, as well. Themed with an engaging city-by-city travel slant, and styled with a very contemporary look, GECKOPOLY had the Company&rsquo;s competitors seeing green!Publisher's Description"
7408,Peanut Butter & Jelly,"Children's Game,Racing","Pattern Recognition,Set Collection","Let's race to see who'll be first to match the pieces and make a peanut butter and jelly sandwich in proper sandwich order. Players use a spinner to acquire sandwich components: bread, peanut butter, and jelly.This is apparently a different game than Peanut Butter & Jelly, which also had an edition from Parker Brothers in 1971."
7502,Transaction,"Economic,Educational","Auction/Bidding,Secret Unit Deployment,Simultaneous Action Selection,Stock Holding","Transaction, the wall street game is the result of over ten years of work. With the game you can simulate conditions as they happen in the actual stock market. This is not just a game of printed cards, spinners, dice and paraphernalia usually found in a game. Actual happenings have been carefully gathered and programmed into a new device which releases authentic stock market data similar to the release of such data by an expensive electronic data processing unit."


In [94]:
text = data.description.iloc[0]
' '.join(re.split(r'(?<=[.:;])\s', text)[:3])

'In Catan (formerly The Settlers of Catan), players try to be the dominant force on the island of Catan by building settlements, cities, and roads. On each turn dice are rolled to determine what resources the island produces. Players collect these resources (cards)&mdash;wood, grain, brick, sheep, or stone&mdash;to build up their civilizations to get to 10 victory points and win the game.&#10;&#10;Setup includes randomly placing large hexagonal tiles (each showing a resource or the desert) in a honeycomb shape and surrounding them with water tiles, some of which contain ports of exchange.'

In [95]:
data['short_desc'] = data.description.apply(lambda x: ' '.join(re.split(r'(?<=[.:;])\s', x)[:4]))

In [163]:
import lxml.html
data['short_desc'] = data['short_desc'].apply(lambda x: lxml.html.fromstring(x).text_content())